In [181]:
import json
import numpy as np
import pandas as pd

In [216]:
# load the data
df = pd.read_csv("station_zip.csv", index_col=None)

/home/kilian/.envs/bittrex/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [217]:
# just need these two columns
df = df[["ZCTA5CE10", "station_id"]].copy()

In [218]:
# count the number of zipcodes each station deals with
df["zipcounts"] = df.groupby('station_id', as_index=False)['station_id'].transform(lambda s: s.count())

In [219]:
# load in the contribution amounts
f = open("dollar_amounts_by_station_id.json")
jlist = []
for line in f.readlines():
    jlist.append(json.loads(line))

In [220]:
amounts = pd.DataFrame(columns=["station_id", "amount"])

# grab the station id and sum over the contributions for each station
ids = [j["stationId"] for j in jlist]
dollas = [np.sum(np.array([entry['amount'] for entry in item['amounts'] if entry["amount"] < 100000])) for item in jlist]

amounts["station_id"] = ids
amounts["amount"] = np.around(dollas, decimals=2)

In [221]:
amounts.amount.values

array([  2.11587065e+06,   9.84296570e+05,   5.24509847e+06, ...,
         7.95348000e+05,   2.76314000e+03,   4.58550000e+04])

In [113]:
amounts["station_id"][amounts.amount == amounts.amount.max()]
[print(j) for j in jlist if j["stationId"] == 6673]

{'amounts': [{'url': 'https://publicfiles.fcc.gov/api/manager/download/1bd4c3a9-5933-2cbb-ca0d-e17b37e3f92d/5ad05b89-5ad5-4e2b-baf7-1bc7969f779a.pdf', 'amount': 4.710395454313986e+24, 'row': 57529}, {'url': 'https://publicfiles.fcc.gov/api/manager/download/1bd4c3a9-5933-2cbb-ca0d-e17b37e3f92d/6f3d9e44-b020-4cf8-b87b-98da87e0ddde.pdf', 'amount': 2610.0, 'row': 229034}, {'url': 'https://publicfiles.fcc.gov/api/manager/download/6a99506e-73a0-8180-5741-83a0a6a6fd23/fd1b7cfc-eb12-4cb7-abf1-f2f51c7e149b.pdf', 'amount': 500.0, 'row': 252542}, {'url': 'https://publicfiles.fcc.gov/api/manager/download/585d7519-5105-5cf2-88c0-a52e3d64f9ee/7cd901c2-004d-47e7-881a-ba5211f98f9f.pdf', 'amount': 480.0, 'row': 252547}, {'url': 'https://publicfiles.fcc.gov/api/manager/download/56cb2c63-4233-f45c-a02a-2bbc16ab2b96/6f3d9e44-b020-4cf8-b87b-98da87e0ddde.pdf', 'amount': 2610.0, 'row': 253223}, {'url': 'https://publicfiles.fcc.gov/api/manager/download/dabecfe0-8eae-a1bd-c22c-b1c07a267ce2/5ad05b89-5ad5-4e2b-b

[None]

In [222]:
totalmoney = amounts.amount.sum()
totalmoney

3148855104.25

In [223]:
# merge the month's amounts to the station-zipcode dataframe
df = df.merge(amounts, how="inner")

In [224]:
# use the zipcode multiplicity of the stations to find the average contribution per zipcode
df["zip_amount"] = df["amount"].values / df["zipcounts"].values

In [225]:
df["zip_amount"] = np.around(df["zip_amount"].values, decimals=2)
df.zip_amount.values

array([   1136.23,    1136.23,    1136.23, ...,  109955.22,  335389.67,
        147434.  ])

In [226]:
# now summing the dollars over zipcodes (contributions from each station)
df["zipsum"] = df.groupby("ZCTA5CE10", as_index=False)["zip_amount"].transform(lambda s: s.sum())

In [227]:
df["zipsum"] = np.around(df["zipsum"].values, decimals=2)
df.zipsum.values

array([   1784.17,  265707.17,   20425.99, ...,  119881.91,  335389.67,
        166173.73])

In [69]:
df.shape

(429628, 6)

In [228]:
# at this point I can get rid of duplicate zipcodes, since station info is irrelevant
zipsumdf = df[["ZCTA5CE10", "zipsum"]]
zipsumdf = zipsumdf.drop_duplicates()
zipsumdf.shape

(33636, 2)

In [229]:

zcdf = pd.read_csv("./zcta_county_rel_10.txt")

zcdf.head()

,ZCTA5,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,601,72,1,72001,18465,7695,165132671,164333375,18570,7744,...,173777444,172725651,99.43,99.37,98.61,98.60,94.77,94.71,95.03,95.14
1,601,72,141,72141,105,49,2326414,2326414,18570,7744,...,298027589,294039825,0.57,0.63,1.39,1.40,0.32,0.35,0.78,0.79
2,602,72,3,72003,41520,18073,83734431,79288158,41520,18073,...,117948080,79904246,100.00,100.00,100.00,100.00,98.95,98.99,70.99,99.23
3,603,72,5,72005,54689,25653,82063867,81880442,54689,25653,...,195741178,94608641,100.00,100.00,100.00,100.00,89.73,90.23,41.92,86.55
4,606,72,93,72093,6276,2740,94864349,94851862,6615,2877,...,94864349,94851862,94.88,95.24,86.56,86.56,100.00,100.00,100.00,100.00


In [230]:
zcdf["STATECOUNTY"] = zcdf.STATE.apply(str).str.zfill(2) + zcdf.COUNTY.apply(str).str.zfill(3)

In [231]:
# find which counties have the max ZCTA area
zcdf['ZCTA_max'] = zcdf.groupby(['ZCTA5'])['ZAREAPCT'].transform(max)

# keep the zipcode-county pairs with max values
zcdf = zcdf[zcdf['ZAREAPCT'] == zcdf['ZCTA_max']]

zcdf.shape

(33121, 26)

In [232]:
# keep the columns we want and rename zcta to have the same name as the original df
zcdf = zcdf[["ZCTA5", "STATECOUNTY"]].copy()
zcdf.rename(columns={'ZCTA5': 'ZCTA5CE10'}, inplace=True)

In [233]:
# now merge on the zipcode column
zipsumdf = zipsumdf.merge(zcdf, how="inner", on="ZCTA5CE10", left_index=False, right_index=False)

In [234]:
zipsumdf.columns

Index(['ZCTA5CE10', 'zipsum', 'STATECOUNTY'], dtype='object')

In [235]:
zipsumdf["county_sum"] = zipsumdf.groupby("STATECOUNTY")["zipsum"].transform(lambda s: s.sum())

In [236]:
zipsumdf["county_sum"] = np.around(zipsumdf["county_sum"].values, decimals=2)
zipsumdf.county_sum.describe()
zipsumdf.shape

(31277, 4)

In [250]:
# now we can just deal with counties
countydf = zipsumdf[["STATECOUNTY", "county_sum"]].copy()
countydf = countydf.drop_duplicates()

In [251]:
countydf.shape

(3150, 2)

In [252]:
countydf["county_sum"] = np.around(countydf["county_sum"].values, decimals=2)
countydf.county_sum.max()

51780975.420000002

In [154]:
countydf.to_csv("county_money.tsv", sep="\t", columns=["STATECOUNTY", "county_sum"], index=False, float_format="%.2f")

In [147]:
np.min(countydf.county_sum)

9.9299999999999997

In [148]:
countydf.county_sum.max()

103362305.15000001

In [83]:
totalmoney = countydf.county_sum.sum()
totalmoney

2939087443.2271957

In [153]:
zipsumdf.to_csv("zipcode_money.tsv", sep="\t", columns=["ZCTA5CE10", "zipsum"], index=False, float_format="%.2f")

In [90]:
countydf.county_sum.min()

9.9347471451876022

In [159]:
# add latitude and longitude to zip file
latlong = pd.read_csv("latlong.dat")

# rename zip column again
latlong.rename(columns={'ZIP': 'ZCTA5CE10'}, inplace=True)

In [160]:
zipsumdf = zipsumdf.merge(latlong, how="inner", on="ZCTA5CE10", left_index=False, right_index=False)

In [161]:
zipsumdf.shape

(31277, 6)

In [162]:
zipsumdf.to_csv("zipcode_money.tsv", sep="\t", columns=["ZCTA5CE10", "LAT", "LNG", "zipsum"], index=False, float_format="%.2f")

In [163]:
countydf.county_sum.max()

103362305.15000001

In [253]:
countydf["logsum"] = np.log10(countydf.county_sum)
countydf.shape

(3150, 3)

In [242]:
countydf.to_csv("county_log_money.tsv", sep="\t", columns=["STATECOUNTY", "logsum"], index=False, float_format="%.2f")

In [281]:
unemployment = pd.read_csv("unemployment.tsv", sep="\t")
unemployment["STATECOUNTY"] = unemployment.id.apply(str).str.zfill(5)
unemployment.shape

(3219, 3)

In [267]:
countydf = pd.merge(unemployment, countydf, how="left", on="STATECOUNTY")
countydf = countydf.fillna(0)
logsumarr = countydf.logsum.values
logsumarr[np.where(logsumarr < 0)] = 0
logsumarr = logsumarr + 1
countydf["logsum"] = logsumarr

In [269]:
countydf.to_csv("county_log_money.tsv", sep="\t", columns=["STATECOUNTY", "logsum"], index=False, float_format="%.1f")

In [272]:
(unemployment.STATECOUNTY.values == countydf.STATECOUNTY.values).all()

True

In [273]:
unemployment.rate.describe()

count    3219.000000
mean        5.423330
std         2.388199
min         1.600000
25%         4.000000
50%         5.000000
75%         6.200000
max        26.400000
Name: rate, dtype: float64

In [274]:
countydf.logsum.describe()

count    3219.000000
mean        6.139114
std         1.222982
min         1.000000
25%         5.673739
50%         6.325073
75%         6.892414
max         8.714170
Name: logsum, dtype: float64

In [282]:
unemployment = unemployment[["STATECOUNTY", "rate"]].copy()
unemployment.rename(columns={'STATECOUNTY': 'id'}, inplace=True)

In [285]:
unemployment.to_csv("unemployment_test.tsv", sep="\t", columns=["id", "rate"], index=False, float_format="%.1f")